# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

623

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

#build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

#print(query_url) #comment to remove api from view

In [12]:
print("Beginning Data Retrival")
print("-----------------------------")

# set up lists to hold reponse info
city_name=[]
lat_list=[]
lng_list=[]
max_temp=[]
humidity=[]
cloudiness=[]
wind_speed=[]
country=[]
date=[]

#counters
index_count=0
set_count=1

# Loop through the list of cities and perform a request for data on each

for index, city in enumerate(cities, start= 1):
    try:
        
        response = requests.get(query_url + city).json()
        city_name.append(response["name"])
        lat_list.append(response["coord"]["lat"])
        lng_list.append(response["coord"]["lon"])
        max_temp.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        cloudiness.append(response["clouds"]["all"])
        wind_speed.append(response["wind"]["speed"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        if index_counter > 49:
            index_counter= 0
            set_count + 1
        else:
            index_count + 1
        print(f"Processing Record {index_count}of set {set_count} : {city}")
    
    except(KeyError, IndexError):
        print("City not found. Skipping...")
    print("-----------------------------")
    print("Data Retrieval Complete")


Beginning Data Retrival
-----------------------------
Processing Record 0of set 1 : mataura
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : rikitea
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : hobart
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : hermanus
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : hithadhoo
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : bela
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : east london
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : yellowknife
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : boma
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : ayagoz
-----------------------------
Data Retrieval Complete
Processing Record 0of s

Processing Record 0of set 1 : paita
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : cap malheureux
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : atuona
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : nova olimpia
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : new norfolk
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : torbay
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : ribeira grande
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : noyabrsk
-----------------------------
Data Retrieval Complete
City not found. Skipping...
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : saint-philippe
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : vila
----------------------------

Processing Record 0of set 1 : aklavik
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : pali
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : prince rupert
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : abeche
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : saint-leu
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : verkhneyarkeyevo
-----------------------------
Data Retrieval Complete
City not found. Skipping...
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : guararapes
-----------------------------
Data Retrieval Complete
City not found. Skipping...
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : leningradskiy
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : itarema
-----------------------------
Data Retri

Processing Record 0of set 1 : shebunino
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : chandur bazar
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : north bend
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : paeroa
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : kamiiso
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : broome
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : bereda
-----------------------------
Data Retrieval Complete
City not found. Skipping...
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : port-gentil
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : oksfjord
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : grindavik
-----------------------------
Data Re

Processing Record 0of set 1 : antofagasta
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : pisco
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : chicama
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : blyth
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : oeiras do para
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : santa rita do passa quatro
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : chama
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : bayaguana
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : sioux lookout
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : maceio
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : swellendam
--------

Processing Record 0of set 1 : westport
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : sao joao da barra
-----------------------------
Data Retrieval Complete
City not found. Skipping...
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : homer
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : hamina
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : plouzane
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : shiraz
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : sosnovoborsk
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : matane
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : atambua
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : koindu
-----------------------------
Data Retrie

Processing Record 0of set 1 : los santos
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : garden city
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : tongliao
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : bagotville
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : taldan
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : rota
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : nahavand
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : obera
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : una
-----------------------------
Data Retrieval Complete
City not found. Skipping...
-----------------------------
Data Retrieval Complete
Processing Record 0of set 1 : moree
-----------------------------
Data Retrieval Complet

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [5]:
#  Get the indices of cities that have humidity over 100%.


In [6]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression